<a id="top"></a>
# HSTaXe Cookbook: Basic Extraction for WFC3/IR 

This notebook contains a step-by-step guide for performing a basic spectral extraction with HSTaXe for G102 or G141 data from WFC3/IR. 

***
## Learning Goals
In this tutorial, you will:

- Organize input data
- Set up HSTaXe and prepare data for extraction
- Learn how to handle different types of background subtraction
- Extract 1-D spectra with a simple box extraction

## Table of Contents

[Introduction](#intro) <br>
[1. Imports](#import) <br>
[2. Setup](#setup) <br>
- [2.1 Drizzling Input Data](#drizzle) <br>
- [2.2 Creating a Catalog with SExtractor](#catalog) <br>

[3. Running HSTaXe](#axe) <br>
- [3.1. Outputs](#out) <br>

[4. Conclusions](#conclusions) <br>
[About this Notebook](#about) <br>
[Citations](#cite) <br>

## Introduction <a id="intro"></a>

[HSTaXe](https://hstaxe.readthedocs.io/en/latest/index.html) is a Python package that provides spectral extraction processes for HST data. Please ensure that you have installed the `hstaxe` module according to the instructions in the documentation, and are running this notebook from the environment created through those instructions.

This notebook will take you through a basic extraction using WFC3/IR grism data. Example data using the G141 grism are available [here](https://stsci.box.com/s/j2ygj4gaqgzmp0b4xcc1h2rszz6cv9wm) if you would like to practice the workflow. You can dowload all required data in the `example_data` subdirectory, and store it within the same parent directory as this notebook.

This notebook will also require configuration files for HSTaXe, which can be downloaded [here](https://stsci.box.com/s/vzkf9x6fmnkjxq6q3v1gbuggk4y3r7kx). These files should then be stored in the `CONF` subdirectory that will be created later in the notebook.

# 1. Imports <a id="import"></a>

For this workflow, we will import the following modules:

- *os*, *glob* and *shutil*, for file handling
- *numpy* for array handling
- *astropy.io.fits* for FITS file handling
- *matplotlib.pyplot* for plotting
- *astrodrizzle* for creating input image mosaics
- *hstaxe.axetasks* for performing the spectral extraction

This notebook will also require a working installation of SExtractor, which will be used to create an input catalog for HSTaXe. Instructions for installing SExtractor are available [here](https://sextractor.readthedocs.io/en/latest/Installing.html).

In [ ]:
%matplotlib inline

import os
import shutil
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
from astropy.table import Table
from drizzlepac import astrodrizzle
from hstaxe import axetasks

## 2. Setup <a id="setup"></a>

We'll start our basic extraction workflow by organizing our input data. A set of example data are available [here](https://stsci.box.com/s/tpbhvrqtbtwod7tr7uijexttoocy4duj) for tutorial purposes.

First, we save the working directory for this notebook.

In [ ]:
cwd = os.getcwd()
print(f'The current directory is: {cwd}')

Next, we'll create directories for our grism and direct images. **HSTaXe will modify our input images in-place, so it is crucial to retain clean versions of them in another location, which will be copied into these directories.** If running this notebook multiple times, run all the lines in the next cell to clear any existing inputs; otherwise, you can run only the uncommented lines.

In [ ]:
os.chdir(cwd)
if os.path.isdir('grism_ims'):
    shutil.rmtree('grism_ims')
if os.path.isdir('direct_ims'):
    shutil.rmtree('direct_ims')
os.mkdir('grism_ims')
os.mkdir('direct_ims')

Now, copy your images to the input directories.

In [ ]:
# src = '/path/to/your/grism/images/*.fits'
src = 'example_data/grism/*.fits'
dst = 'grism_ims/'
for f in glob.glob(src):
    shutil.copy(f, dst)

# src = '/path/to/your/direct/images/*.fits'
src = 'example_data/direct/*.fits'
dst = 'direct_ims/'
for f in glob.glob(src):
    shutil.copy(f, dst)

## 2.1 Drizzling the Input Data<a id="drizzle"></a>
The next step is to drizzle the grism images. We'll need a list of the image names to feed to AstroDrizzle. After that, we'll do the same for the direct images, but use the drizzled grism image as a reference, which will ensure proper registration between the data. HSTaXe will use these linked drizzle images to locate spectral traces based on the positions of sources in the direct images.

In [ ]:
# Creat list file using images in grism directory
os.chdir('grism_ims')

lis = open('grism.lis', 'w')
for f in sorted(os.listdir('.')):
    if os.path.splitext(f)[1]=='.fits':
        lis.write(f)
        lis.write('\n')
lis.close()

!cat grism.lis

In [ ]:
# Drizzle grism images. If only using one input image, set blot, median, driz_cr to False
astrodrizzle.AstroDrizzle('@grism.lis', output='grism', 
                          build=True, blot=True, median=True, driz_cr=True)

In [ ]:
# List file for direct images
os.chdir(cwd)
os.chdir('direct_ims')

lis = open('direct.lis', 'w')
for f in sorted(os.listdir('.')):
    if os.path.splitext(f)[1]=='.fits':
        lis.write(f)
        lis.write('\n')
lis.close()

!cat direct.lis

In [ ]:
# Drizzle direct images, using drizzled grism mosaic as a reference to ensure proper registration.
# If only using a single direct image, set the CR rejection keywords to False.
# If your input images were flc images rather than flt images, change the extension to 'drc.fits'
ref = '../grism_ims/grism_drz.fits[1]'
astrodrizzle.AstroDrizzle('@direct.lis', output='direct', build=True, 
                          final_wcs=True, driz_sep_wcs=True,
                          driz_sep_refimage=ref, final_refimage=ref,
                          blot=True, median=True, driz_cr=True)

Your grism and direct images should now be aligned. For the WFC3/IR grisms, there should be very little vertical offset between the positions of sources in the direct image and their correspondents in the grism image. We'll perform a quick visual check here:

In [ ]:
os.chdir(cwd)
fig, axs = plt.subplots(1, 2, figsize=(8,8))

d = fits.getdata('direct_ims/direct_drz.fits', 1)
im1 = axs[0].imshow(d, origin='lower')

d = fits.getdata('grism_ims/grism_drz.fits', 1)
im2 = axs[1].imshow(d, origin='lower')

# Adjust image levels here
im1.set_clim(0, 0.2)
im2.set_clim(0, 0.2)

plt.tight_layout()

## 2.2 Creating a Catalog with SExtractor<a id="catalog"></a>

With our drizzle images created, we now need a catalog of sources in the direct image. Creating this catalog requires using SExtractor, which may need to be installed separately; please refer to the link in the Imports section for instructions on how to do this.

HSTaXe will look for a highly specific format in the catalog, and does not always give clear error messages when something within the catalog is awry. Please follow the next steps carefully:

1. Copy the drizzled direct image into the `sextractor` directory, which contains HSTaXe-appropriate configuration files for SExtractor.

In [ ]:
shutil.copy('direct_ims/direct_drz.fits', 'sextractor/')

2. With SExtractor installed, run the following command from within the `sextractor` directory that came with this notebook:
    `sex -c aXe.sex direct_drz.fits[1] -DETECT_THRESH 5 -MAG_ZEROPOINT 26.4525`

    Note that the value for the `DETECT_THRESH` keyword, which sets the minimum value for pixels to be considered, may be changed appropriately for your data.

In [ ]:
os.chdir('sextractor')

detect_thresh = 5
cl_input = f'sex -c aXe.sex direct_drz.fits[1] -DETECT_THRESH {detect_thresh} -MAG_ZEROPOINT 26.4525'

In [ ]:
os.system(cl_input)

In [ ]:
# Copy the catalog to the direct image directory
os.chdir(cwd)
# shutil.copy('sextractor/aXe.cat', 'direct_ims')

# Copy the example catalog to the direct image directory:
shutil.copy('example_data/aXe.cat', 'direct_ims')

3. Examine the catalog. The "MAG_ISO" column must be renamed to "MAG_F####" for it to be correctly read in by HSTaXe, with "####" being replaced by a number, nominally the pivot wavelength of the direct image filter in nm (e.g. 1392 for F140W). Any lines containing clearly spurious detections, such as those with magnitudes of $\pm$99, should also be removed. **Note**: These edits must be done manually as astropy only supports reading of SExtractor-formatted tables, not writing.

    Additionally, locate the lines containing the sources whose spectra you want to extract and note the line number from the NUMBER column. This will be used later to identify the BEAM number for your object in the output files.

In [ ]:
cat = Table.read('direct_ims/aXe.cat', format='ascii.sextractor')
cat

# Rename the MAG_ISO column and remove spurious rows manually!

In [ ]:
# Use this cell to filter your catalog for your sources
# In this example, we sort to identify the brightest sources
cat.sort('MAG_F1392')
cat

# 3. Running HSTaXe<a id="aXe"></a>

With the catalog generated, we can now move on to working with HSTaXe. We'll set up some directories and environment variables that point to them, while clearing out any previous data or outputs in these directories. We won't clear the `CONF` directory, which contains configuration files with information that HSTaXe will use to locate the spectra in our grism images. We'll also copy our data into the fresh `DATA` directory.

In [ ]:
os.chdir(cwd)

# DATA
if os.path.isdir('DATA'):
    shutil.rmtree('DATA')
os.mkdir('DATA')
os.environ['AXE_IMAGE_PATH'] = './DATA/' 

src = 'direct_ims/*flt.fits'
dst = 'DATA'
for f in glob.glob(src):
    shutil.copy(f, dst)
src = 'grism_ims/*flt.fits'
for f in glob.glob(src):
    shutil.copy(f, dst)

In [ ]:
# CONF
os.environ['AXE_CONFIG_PATH'] = './CONF/'

In [ ]:
# OUTPUT
if os.path.isdir('OUTPUT'):
    shutil.rmtree('OUTPUT')
os.mkdir('OUTPUT')
os.environ['AXE_OUTPUT_PATH'] = './OUTPUT/'

Next, we define the field-of-view boundaries for the detector. We'll pass this information to the `iolprep` task, which will let it include in the input object lists (IOLs) it generates, objects whose direct image locations fall outside of the chip but whose spectral traces do fall onto the chip.

For WFC3/IR, the [left, right, top, bottom] extensions, in pixels, are [183, 85, 50, 50]

In [ ]:
FOV = '183,85,50,50'

Now we'll run `iolprep` to generate our IOLs, which are object catalogs for each individual direct image, from the drizzled direct image and its catalog.

In [ ]:
os.chdir(cwd)
os.chdir('direct_ims')

axetasks.iolprep(drizzle_image = 'direct_drz.fits',
                input_cat = 'aXe.cat',
                dimension_in = FOV)

In [ ]:
# Copy the IOLs to the aXe DATA directory
os.chdir(cwd)
for f in glob.glob('direct_ims/*_?.cat'):
    shutil.copy(f, 'DATA')

The last step before extracting our spectra is to generate a file which contains on each line the names of a grism image, IOL name, and associated direct image. This is best done manually, to ensure that each grism image lines up with the appropriate direct image. For the example data, a file called `example.lis` is provided.

With this list, we run `axeprep`, which prepares the individual images for spectral extraction. This step is also responsible for perfoming a global background subtraction, if desired. This is controlled by the `backgr` keyword.

Note that the `configs` and `backims` keywords should be matched to the data you have. E.g., if you are working with G102 data with direct images in F098M, the config file should be `G102.F098M.V4.31.conf`. If you are not performing a global background subtraction, `backims` is not required, but should be matched to the correct grism if you are.

In [ ]:
# Uncomment below to copy and rename the example list file
# os.chdir(cwd)
# shutil.copy('example_data/example.lis', '.')
# os.rename('example.lis', 'aXe.lis')

In [ ]:
os.chdir(cwd)

axetasks.axeprep(inlist='aXe.lis',
                     configs='G141.F140W.V4.31.conf',
                     backims='WFC3.IR.G141.sky.V1.0.fits',
                     backgr=False,
                     norm=False,
                     mfwhm=3.0)

The last HSTaXe task to run is `axecore`, which performs the actual extraction and generates output files. Again, the configuration file and sky background arguments should be matched to the spectral elements used for your data.

Local background subtraction is also performed by this step, if desired. The following keywords are critical for local background:

* `back`: This argument is the flag to trigger local background subtraction.
* `np`: Defines the number of pixels on either side of the spectral trace (beam) used to calculate the local background from.
* `interp`: Sets the interpolation method for the local background (-1=median, 0=mean, ≥1=nth order polynomial)
* `backfwhm`: The FWHM specifying the width of the background pixel extraction table

More information on background handling, both global and local, with `HSTaXe` can be found in the documentation [here](https://hstaxe.readthedocs.io/en/latest/hstaxe/description.html#sky-background).

In [ ]:
axetasks.axecore('aXe.lis',
                 'G141.F140W.V4.31.conf',
                 fconfterm='WFC3.IR.G141.sky.V1.0.fits',
                 extrfwhm=4.,
                 drzfwhm=3.,
                 backfwhm=5.,
                 orient=False,
                 back=False,
                 weights=True,
                 slitless_geom=True,
                 cont_model='gauss',
                 sampling='drizzle',
                 exclude=True)

## 3.1. Outputs<a id="out"></a>

Each grism input file will have several corresponding output files. For now, let's take a look at the STP files, which contain 2D "stamps" of the extracted spectral traces; and the SPC files, which contain our 1D extracted spectra.

We'll need the line numbers from the original source catalog we generated to identify the BEAM number for the objects whose spectra we want. For the example data, we'll use the bright star in the bottom-left of the direct images, which is on line 239 in the provided catalog.

We'll first look at the STP files:

In [ ]:
!ls OUTPUT/*SPC.fits
!ls OUTPUT/*STP.fits

In [ ]:
beam = '239'

fig, axes = plt.subplots(4, 1, figsize=(12,9))

for i, f in enumerate(glob.glob('OUTPUT/*STP.fits')):
    with fits.open(f) as hdul:
        d = hdul[f'BEAM_{beam}A'].data
        im = axes[i].imshow(d, origin='lower')
        axes[i].set_title(os.path.basename(f))
        im.set_clim(0, 5.0)
        
plt.tight_layout()

And now, the SPC files:

In [ ]:
beam = '239'

fig, axes = plt.subplots(4, 1, figsize=(12,12), sharex=True)

for i, f in enumerate(glob.glob('OUTPUT/*2.SPC.fits')):
    with fits.open(f) as hdul:
        d = hdul[f'BEAM_{beam}A'].data
        wl = d['LAMBDA']
        flux = d['FLUX']
        error = d['FERROR']
        xrange = (wl>11500) & (wl<16000)
        axes[i].errorbar(wl[xrange],flux[xrange],error[xrange])
        axes[i].set_title(os.path.basename(f))
        axes[i].set_ylabel(r'Flux ($erg/s/cm^2/\AA/s$)')        
axes[3].set_xlabel(r'Wavelength ($\AA$)')
        
plt.tight_layout()

# 4. Conclusions <a id="conclusions"></a>

Thank you for walking through this spectral extraction workflow. You should now be able to perform a basic extraction on WFC3/IR data using HSTaXe.

For additional information on the WFC3 grisms, please visit the [grism resources](https://www.stsci.edu/hst/instrumentation/wfc3/documentation/grism-resources) and [grism data analysis](https://www.stsci.edu/hst/instrumentation/wfc3/documentation/grism-resources/grism-data-analysis) webpages.

Further workflow cookbooks are available on the [HSTaXe GitHub](https://github.com/spacetelescope/HSTaXe), including a more advanced IR extraction, and a UVIS extraction. For detailed information on HSTaXe, please visit the [documentation webpage](https://hstaxe.readthedocs.io/en/latest/index.html).

**Congratulations, you have completed the notebook.**

## About this Notebook <a id="about"></a>

**Author:** Aidan Pidgeon, WFC3 Instrument Team

**Special Thanks to:** 
 - Dr. Nor Pirzkal, for creating the original workflow that was adapted into this notebook
 - Ricky O'Steen and Duy Nguyen, for their fantastic work in updating the HSTaXe module
 - Debopam Som and Benjamin Kuhn, for support in testing the HSTaXe workflow

**Updated On:** 2022-01-06

## Citations <a id="cite"></a>

If you use `astropy` for published research, please cite the
authors. Follow this link for more information about citing
`astropy`:

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)